In [243]:
import pandas as pd
import csv
import math
import networkx as nx 
import heapq
import random 
# numpy pour dijkstra 
import numpy as np
from numpy import Inf

#Widget
import ipywidgets as widgets
from IPython.display import display

#Map interactive
import folium
from folium.plugins import MarkerCluster

#Map pour connection
import matplotlib.pyplot as plt
%matplotlib inline

#Calcul great circle
import pyproj

import tkinter
#import os
#import flightradar24
from branca.element import Figure
import igraph as ig



In [244]:
#pip install markupsafe==2.1.1

In [245]:
#pip install pandas
#pip install networkx
#pip install ipywidgets
#pip install folium
#pip install matplotlib
#pip install pyproj
#pip install nbinteract
#pip install scipy
# pip3 install nbdev
#python -m pip install markupsafe==2.0.1

In [246]:
#python -m pip install markupsafe==2.0.1

In [247]:
#pip install voila
# pip install networkx
# pip install pyproj

# pyhton -m pip install package 

In [248]:
#chemin="../data/"
chemin="../../data/"


#df_aeroports = pd.read_csv(chemin + "aeroportsANLO.csv")
#df_zones = pd.read_csv(chemin + "zonesANLO.csv")
df_vols = pd.read_csv(chemin + "routesANLO.csv")
#df_compagnies = pd.read_csv(chemin + "compagniesANLO.csv")

In [249]:
dict_compagnies = {}

with open(chemin+'compagniesANLO.csv', mode='r') as inp:
    reader = csv.reader(inp)
    next(reader, None)
    headers = ['Name','Country']
    dict_compagnies = {rows[2]:dict(zip(headers,[rows[0],rows[3]])) for rows in reader}


In [250]:
dict_aeroports = {}

with open(chemin+'aeroportsANLO.csv', mode='r') as inp:
    reader = csv.reader(inp)
    headers = next(reader, None)
    dict_aeroports = {rows[1]:dict(zip(headers,rows[0:8])) for rows in reader}

dict_aeroports['JFK']
dict_aeroports['EWR']
dict_aeroports['LGA']
#DistGrandCercleICAO('JFK', 'EWR') < 50 

{'Name': 'New York LaGuardia Airport',
 'Iata': 'LGA',
 'Icao': 'KLGA',
 'Lat': '40.777248',
 'Lon': '-73.872597',
 'Country': 'United States',
 'Alt': '21',
 'City': 'New York'}

In [251]:
dict_pays = {}

with open(chemin+'countries.csv', mode='r') as inp:
    reader = csv.reader(inp)
    headers = next(reader, None)
    dict_pays = {rows[1]:rows[0] for rows in reader}

#dict_pays

In [252]:
df_vols = df_vols[["Depart","Arrivee","Code_ICAO"]].dropna()

In [253]:
''''
#Nettoyage de la df_vols
for _, ligne in df_vols.iterrows():
    if not(ligne.Depart in dict_aeroports):
        df_vols.drop(df_vols[df_vols['Depart'] == ligne.Depart].index, inplace=True)
    
    if not(ligne.Arrivee in dict_aeroports):
        df_vols.drop(df_vols[df_vols['Arrivee'] == ligne.Arrivee].index, inplace=True)
    ''' 

"'\n#Nettoyage de la df_vols\nfor _, ligne in df_vols.iterrows():\n    if not(ligne.Depart in dict_aeroports):\n        df_vols.drop(df_vols[df_vols['Depart'] == ligne.Depart].index, inplace=True)\n    \n    if not(ligne.Arrivee in dict_aeroports):\n        df_vols.drop(df_vols[df_vols['Arrivee'] == ligne.Arrivee].index, inplace=True)\n    "

In [254]:
def coordDepuisAeroport(iata : str):
    '''
    Fonction : Recherche les coordonnées GPS d'un aéroport
    Retour : lat : float ,lon : float --> Latitude & longitude de l'aéroport
    '''
    lat = float(dict_aeroports[iata]['Lat'])
    lon = float(dict_aeroports[iata]['Lon'])
    return lat,lon

def DistGrandCercle(lat1 : float, lon1 : float, lat2 : float, lon2 : float):
    #Conversion en radian
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])

    #Delta des coordonnées
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    #Rayon terrestre (en km)
    r = 6367.0

    #Formule d'Haversine
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    return 2 * r * math.asin(math.sqrt(a))


def DistGrandCercleICAO(icao1 : str, icao2 : str):
    #Récupération des coordonnées des aéroports
    latDep, lonDep = coordDepuisAeroport(icao1)
    latArr, lonArr = coordDepuisAeroport(icao2)

    #On calcule la distance les séparant,                                                     
    return DistGrandCercle(latDep,lonDep,latArr,lonArr)

def Sqr(a):
    return a*a

def Dist(x1,y1,x2,y2):
    return math.sqrt((y2-y1)**2+(x2-x1)**2)

In [255]:
import json 
import pickle

In [256]:
'''
dict_route = {}

g = df_vols.groupby(['Depart','Arrivee'])
taille_groupe = g.size().items()

for liaison, nbrVols in taille_groupe:
    #Si la liaison est déjà dans le dict, on incrémente le nbr de vols
    if(frozenset(liaison) in dict_route):
        dict_route[frozenset(liaison)]['Vols'] += nbrVols
    #Sinon, on crée une nouvelle liaison
    else:
        dict_route[frozenset(liaison)] = {'Dist' : DistGrandCercleICAO(liaison[0],liaison[1]), 'Vols' : nbrVols}


a_file = open("dict_route.pkl", "wb")
pickle.dump(dict_route, a_file)
a_file.close()
'''

'\ndict_route = {}\n\ng = df_vols.groupby([\'Depart\',\'Arrivee\'])\ntaille_groupe = g.size().items()\n\nfor liaison, nbrVols in taille_groupe:\n    #Si la liaison est déjà dans le dict, on incrémente le nbr de vols\n    if(frozenset(liaison) in dict_route):\n        dict_route[frozenset(liaison)][\'Vols\'] += nbrVols\n    #Sinon, on crée une nouvelle liaison\n    else:\n        dict_route[frozenset(liaison)] = {\'Dist\' : DistGrandCercleICAO(liaison[0],liaison[1]), \'Vols\' : nbrVols}\n\n\na_file = open("dict_route.pkl", "wb")\npickle.dump(dict_route, a_file)\na_file.close()\n'

In [257]:
a_file = open("dict_route.pkl", "rb")
dict_route = pickle.load(a_file)

In [258]:
def filtreDictAeroports(colonne : str, valeur : str):
    '''
    Fonction : Filtre les clés du dictionnaire en fonction d'une valeur 
    présente dans les valeurs associées.
    Return : Liste de code ICAO des aéroports
    '''
    Listtmp = []
    for (key, value) in dict_aeroports.items():
        if value[colonne] == valeur:
            Listtmp.append(key)
        
        if valeur == '':
            Listtmp.append(value[colonne])
    return Listtmp

def filtreDictCompagnies(colonne : str, valeur : str):
    '''
    Fonction : Filtre les clés du dictionnaire en fonction d'une valeur 
    présente dans les valeurs associées.
    Return : Liste de code ICAO des compagnies
    '''
    Listtmp = []
    for (key, value) in dict_compagnies .items():
        if value[colonne] == valeur:
            Listtmp.append(key)
            
        if valeur == '':
            Listtmp.append(value[colonne])
    return Listtmp

In [259]:
def correspondanceAirport(code1 : str, code2 : str):
    '''
    fonction qui filtre le df_vols pour obtenir les correspondances
    retour : list des airports codes 
    '''
    r1 = df_vols[df_vols['Depart'] == code1]['Arrivee'].to_list()
    r2 = df_vols[df_vols['Arrivee'] == code2]['Depart'].to_list()

    return list(set(r1).intersection(r2))

In [260]:
def memeContinent(str_air1, str_air2):
    '''
    fonction qui retourne vrai si les deux sont sur le meme continent 
    et faux sinon
    '''
    c1 = dict_aeroports[str_air1]['Country']
    c2 = dict_aeroports[str_air2]['Country']
    
    return c1 == c2
    

def nettoyageList(str_list, str_airStart, str_airEnd):
    '''
    fonction qui supprimer les occurences dans le liste qui ne sont pas sur le 
    meme continent que les airport de depart et d'arrive . 
    '''
    
    if memeContinent(str_airStart, str_airEnd):
        #print('Les airports sont sur le même continent ')
        compteur = 0 ; 
        for element in str_list:
            if not memeContinent(str_airStart, element):
                str_list.remove(element)
                compteur = compteur + 1 
    
    #print('On a enleve {c} airports du list'.format(c = compteur))
    return str_list  
    

In [261]:
def creationEdgesConnection(graphe : nx.Graph, list_reference : list, code1 : str, code2 : str, list_connections_nettoye):
    '''
    fonction qui genere des edges pour notre graphe avec les weights. On appele seulement cette
    focntion si on a le list_connections (qui sont les airports entre code1 et code2)
    retour : list_ref (qui a les index qui corresponds au nodes du graph)
    '''
        
    # pour gerer les correspondances 
    if list_reference.count(code1) == 0: 
        list_reference.append(code1)
        graphe.add_node(list_reference.index(code1))   
    if list_reference.count(code2) == 0: 
        list_reference.append(code2)
        graphe.add_node(list_reference.index(code1))
    
    for i in list_connections_nettoye :
        if list_reference.count(i) == 0 : 
            list_reference.append(i)   
            graphe.add_node(list_reference.index(i))  
    
    index1 = list_reference.index(code1) 
    index2 = list_reference.index(code2)
    
    list_edges = list(map(lambda x :
        (index1, list_reference.index(x), int(DistGrandCercleICAO(code1, x))),
        list_connections_nettoye))
    
    list_edges.extend(list(map(lambda y : 
        (list_reference.index(y), index2, int(DistGrandCercleICAO(y, code2))),
        list_connections_nettoye)))
   
   
    graphe.add_weighted_edges_from(list_edges)
    return list_reference

In [262]:
def creationEdgesAucunConnection(graphe : nx.Graph, list_reference : list, code1 : str, code2 : str):
    print(code1, code2)
    posCoteStart = df_vols[df_vols['Depart'] == code1]['Depart'].to_list()
    posCoteEnd = df_vols[df_vols['Arrivee'] == code2]['Depart'].to_list()
    
    if posCoteStart == []: print('Dans notre database, il y a pas des vols qui part de {} .'.format(code1)) ; return False
    if posCoteEnd == []: print('Dans notre database, il y a pas des vols qui arrive sur {} .'.format(code2)); return False
    
    # des le moment que il trouve une chemin pour reliee les deux airports il le fait 
    for i_code in posCoteStart:
        for j_code in posCoteEnd: 
            # verifie si on a une connection 
            tmp = correspondanceAirport(i_code, j_code)
            # si on a une conenction 
            if len(tmp) > 0 :
                # on ajoute les premiere elements 
                list_reference.extend([code1, code2, i_code, j_code])
                edgeStart = (list_reference.index(code1), list_reference.index(i_code), int(DistGrandCercleICAO(code1, i_code)))
                edgeEnd = (list_reference.index(j_code), list_reference.index(code2), int(DistGrandCercleICAO(j_code, code2)))
                
                graphe.add_weighted_edges_from([edgeStart, edgeEnd])
                return creationEdgesConnection(graphe, list_reference, i_code, j_code, tmp)
                 
            
    print('Aucun possiblite pour relie des airports avec trois correspondances (qui est notre max predifini')

In [263]:
def creationEdges(graphe : nx.Graph, list_reference : list, code1 : str, code2 : str):
    '''
        fonction qui crée à partir d'une liste d'aéroports un list des edges d'une graphe (avec le weight)
        et les ajouts dans le graph donne en parametre 
        retour : list des edges ajoute a la graphe [(start , end, weight), (start, end ,weight ) .. ]
    '''
    # dans le listReference les airport recois un index qui corresponds au identifiant de graphe 
    list_connections = correspondanceAirport(code1, code2)
    list_connections_nettoye = list_connections
    
    if(len(list_connections) > 0 ):
      return creationEdgesConnection(graphe, list_reference, code1, code2, list_connections_nettoye)
    else :
        return print("let op functie"); creationEdgesAucunConnection(graphe, list_reference, code1, code2)

In [264]:
def dessinerGraphe(graphe, weighted : bool = False):
    fig = plt.figure(); 
    fig.set_size_inches(15, 15)
    fig.patch.set_facecolor('white')
 
    pos = nx.fruchterman_reingold_layout(graphe)
    nx.draw_networkx_nodes(graphe, pos, node_size = 200)
    nx.draw_networkx_edges(graphe,pos, edgelist= graphe.edges(), edge_color='black')
    if graphe.number_of_nodes() < 2000:
        nx.draw_networkx_labels(graphe, pos)
    plt.autoscale()
   
    if(weighted): nx.draw_networkx_edge_labels(graphe, pos) # afficher le weight 

    plt.show()

In [265]:
def dijkstra(graphe : nx.Graph, start : int, end : int):
    '''
        fonction qui utilise l'algorithme de Dijkstra pour calcule le plus court
        chemin et calcule le distance
        retour \: dict{\'distance\' : ... , \'path\' : ...}
    '''
    try :
        tmp1 = nx.dijkstra_path(graphe, start,end)
        dist = nx.dijkstra_path_length(graphe, start, end)
        return {'distance' : dist, 'path' : tmp1}
    except nx.NetworkXNoPath:
        print('no path (dijkstra)')
        return -1 
        
    

In [266]:
def nombreDeCorrespondances(g : nx.Graph, codeStart, codeFin):
    listRetour = creationEdges(g, [], codeStart, codeFin)
    ## was not  
    if listRetour == [] : 
        
        print('check in nombreDeCorrespondance Edges : ', len(listRetour), g.number_of_edges())
        
        print('check in nombreDeCorrespondance: ', len(listRetour), g.number_of_nodes())
            
        if g.has_edge(listRetour.index(codeStart), listRetour.index(codeFin)): g.remove_edge(listRetour.index(codeStart), listRetour.index(codeFin))
        if g.has_edge(listRetour.index(codeFin), listRetour.index(codeStart)): g.remove_edge(listRetour.index(codeFin), listRetour.index(codeStart))
        
        return listRetour

    return listRetour

In [267]:
def traitementRequest(aeroport_start : str, aeroport_fin : str, g : nx.Graph = nx.Graph()):
    '''
        fonction qui fait le creation de graphe et qui transofrme les nom de airports en node de graphes
        retour : graphe, list_ref (qui est le lien entre le nombr de nodes et le code de airports)
    '''
    aeroport_codeS = filtreDictAeroports('Name', aeroport_start)[0]
    aeroport_codeE = filtreDictAeroports('Name', aeroport_fin)[0] 
    list_ref = [] 
    list_ref = creationEdges(g, list_ref, aeroport_codeS, aeroport_codeE)
    
    return g ,list_ref
    
def distancePlusCourte(graphe: nx.Graph, list_ref : list, aeroport_codeS, aeroport_codeE):
    '''
        fonction qui trouve le chemin de start aux fin avec le moins de distance possible 
        retour : dictionnaire en forme de {'distance' : .. , 'path' : [ path1, path2 ]}
    '''
    resultat = dijkstra(graphe, list_ref.index(aeroport_codeS), list_ref.index(aeroport_codeE))
    if(resultat == -1 ):
        print('No path (distance plus courte)')
        return -1 
    else : 
        resultat['path'] = list(map(lambda x : list_ref[x], resultat['path']))
        return resultat 

def routeAlternatives(graphe : nx.Graph, list_ref : list, int_start : int, int_end : int):
    '''
        fonction qui trouve tous les chemins de start aux fin avec le meme nombr de correspondances 
        que le route plus courte. 
        retour : dictionnaire en forme de {'nbRoute' : .. , 'path' : [ path1, path2 ]}
    '''

    r = list(nx.all_shortest_paths(graphe,source=int_start,target=int_end))  
    
    tmp = []; 
    for i in r: 
        tmp.append(list(map(lambda x : list_ref[x], i)))
        
    return {'nbRoute' : len(r), 'path' : tmp} 


In [268]:
# copie de loris 
def vectGrandCercle(latDep : float, lonDep : float, latArr : float, lonArr : float):
    '''
    Fonction : Calcul un semble de coordonnées permettant de tracer un grand cercle terrestre
    Return : 2 listes de coordonnées
    '''
    g = pyproj.Geod(ellps='WGS84')
    (az12, az21, dist) = g.inv(lonDep, latDep, lonArr, latArr)

    # Coordonnée avec des segments <= 100 km
    lonlats = g.npts(lonDep, latDep, lonArr, latArr,
                    1 + int(dist / 100000))

    #Mise en forme
    v1 = []
    v1.append([latDep,lonDep])
    v2 = []
    _lon = lonDep
    _lat = latDep
    horscadre = False
    for lon, lat in lonlats:
        if(not(horscadre) and Dist(_lat,_lon,lat,lon) < 4):        
            v1.append([lat,lon])
        else:
            horscadre = True
            v2.append([lat,lon])
        _lon = lon
        _lat = lat
    
    if(horscadre):
        v2.append([latArr,lonArr])
    else:
        v1.append([latArr,lonArr])

    return v1,v2

In [269]:
def creation_lines(path : list, Line_group, c):
    '''
    fonction qui cree les lignes pour le folium map
    path c'est le chemin, line_group c'est le group commun, c c'est le couleur de ligne 
    '''
    for element in range(len(path)-1) :  
        start = path[element]
        end = path[element+1]
        latDep, lonDep = coordDepuisAeroport(start)
        latArr, lonArr = coordDepuisAeroport(end)

        #Calcul des chemins à afficher
        v2 = []
        v1,v2 = vectGrandCercle(latDep,lonDep,latArr,lonArr)
        
        col_ligne = c
        col_marker = c 
        
        #-------------------
        folium.PolyLine(locations=v1,weight=3, color=col_ligne).add_to(Line_group)
        if(len(v2) != 0):
            folium.PolyLine(locations=v2,weight=3, color=col_ligne).add_to(Line_group)

        folium.CircleMarker(location = [latArr, lonArr],
                        radius = 1, 
                        color = col_marker, 
                        tooltip=dict_aeroports[end]['City'],
                        popup="<b>Nom : </b>" + dict_aeroports[end]['Name'] + """<br />
                                <b>Code :</b> """ + dict_aeroports[end]['Iata'] + """<br />
                                <b>Coordonnées :</b> (""" + str(dict_aeroports[end]['Lat']) + ";" + str(dict_aeroports[end]['Lon']) + """)<br />
                                <b>City : </b>""" + str(dict_aeroports[end]['City']) + """<br />
                                <b>Pays : </b>""" + dict_aeroports[end]['Country']).add_to(Line_group)

In [336]:
from IPython.core.display import HTML
from ipywidgets import Output

In [348]:
outputMAP = Output()

def map_correspondance(collection_path):
    '''
    fonction qui gere le ajoute de chaque ligne pour tous les chemins qui sont dans le collection path 
    collection sous la forme de {0 : {'distance' : .. , 'path' : []}, 1 : {..} , etc}
    '''
    
    m = folium.Map(location=[45.763420, 4.834277], zoom_start=2)
    
    # 1 group de line 
    Line_group = folium.FeatureGroup(name = "Tous")

    # generer les differents couleurs pour les differents chemins 
    longeur = 0 
    if isinstance(collection_path, dict) : longeur = len(collection_path.keys())
    if isinstance(collection_path, list) : longeur = len(collection_path)
    rand_colors = []
    for _ in range(longeur):
        rand_colors.extend( ["#"+''.join([random.choice('ABCDEF0123456789') for i in range(6)])] ) 
    
    # ajoute le marker du debut 
    air_start = str
    if isinstance(collection_path, dict) : air_start = collection_path[0]['path'][0]
    if isinstance(collection_path, list) : air_start = collection_path[0][0]
    latStart, lonStart = coordDepuisAeroport(air_start)
    
    folium.CircleMarker(location = [latStart, lonStart],
                        radius = 1, 
                        color = '#000000', 
                        tooltip=dict_aeroports[air_start]['City'],
                        popup="<b>Nom : </b>" + dict_aeroports[air_start]['Name'] + """<br />
                                <b>Code :</b> """ + dict_aeroports[air_start]['Iata'] + """<br />
                                <b>Coordonnées :</b> (""" + str(dict_aeroports[air_start]['Lat']) + ";" + str(dict_aeroports[air_start]['Lon']) + """)<br />
                                <b>City : </b>""" + str(dict_aeroports[air_start]['City']) + """<br />
                                <b>Pays : </b>""" + dict_aeroports[air_start]['Country']).add_to(Line_group)
    
    
    if isinstance(collection_path, dict) : 
        # iteration sur le dict pour ajouter chaque chemin 
        for key, contenu in collection_path.items() : 
            # une couleur par chemin 
            couleur = rand_colors.pop()
            # creation de poly lignes 
            creation_lines(contenu['path'], Line_group, couleur)
        
        
    if isinstance(collection_path, list):
        for contenu in collection_path : 
            # une couleur par chemin 
            couleur = rand_colors.pop()
            # creation de poly lignes 
            creation_lines(contenu, Line_group, couleur)
        
    #Ajout les lines à la carte
    Line_group.add_to(m)
    
    fig = Figure(width='auto', height = 'auto', ratio='50%' )
    fig.add_child(m)

    with outputMAP: 
        display(fig)

    

In [271]:
## marche seulement pour les petit relations 
def routeSelonNombreCorrespondances(graphe : nx.Graph, nombre : int, start, end):
    '''
        fonction qui trouve tous les chemins de start aux fin avec 'nombre' correspondances
        retour : dictionnaire en forme de {'nbRoute' : .. , 'path' : [ path1, path2 ]}
    '''
    r2 = list(nx.all_simple_paths(graphe,source=start,target=end))    
    r3 = list(filter(lambda x : len(x) == nombre, r2))
    
    return {'nbRoute' : len(r3), 'path': r3}

def routeSelonNombreCorrespondanceLessThen(graphe : nx.Graph, nombre : int, start, end):
    '''
        fonction qui retourne tous les routes/correspondances avec un nombre de correspondances <= nombre 
        retour : un dictionnaire { nombre : {'nbRoute' : .. , 'path' : [ path1, path2 ]}, nombre2 : {..}}
    '''
    r1 = list(nx.all_simple_paths(graphe,source=start,target=end))    
    r2 = list(filter(lambda x : len(x) <= nombre, r1))
    
    return {'nbRoute' : len(r2), 'path': r2}


In [272]:
array_arrivee = []
array_depart = [] 
for key in dict_route:

    if len(key) == 2 :
        depart, arrive = key 
        
        array_arrivee.append(arrive)
        array_depart.append(depart)

list_arrivee = np.unique(array_arrivee)
list_depart = np.unique(array_depart)

list_arrivee = list(map(lambda x : dict_aeroports[x]['Name'], list_arrivee))
list_depart = list(map(lambda x : dict_aeroports[x]['Name'], list_depart))

list_arrivee = sorted(list_arrivee)
list_depart = sorted(list_depart)


In [274]:
graph_totale = nx.Graph()
list_airports_codes = []; 

compteur = 0 
list_debug = [] 
for key in dict_route:

    if len(key) == 2 :
        start, end = key 
        compteur = compteur + 1
        
        if list_airports_codes.count(start) == 0:
            list_airports_codes.append(start)

        if list_airports_codes.count(end) == 0:
            list_airports_codes.append(end)
            
        index_start = list_airports_codes.index(start); 
        index_end = list_airports_codes.index(end); 
        distance = dict_route[frozenset((start,end))]['Dist']
        
        graph_totale.add_node(index_start)
        graph_totale.add_node(index_end)
        
        graph_totale.add_edge(index_start, index_end, weight = distance)
    else: list_debug.append(key)

nb_keys = len(dict_route.keys())
#print('Nombre de keys dans le dict_route : ', print(nb_keys))
debug = nb_keys - compteur
#print('Nombre de keys qui ne sont pas ajoute : ', debug , ', voici le tableau avec le keys qui ne sont pas ajoutee')
#print(list_debug)

In [275]:
def supprimerJusqauNombre(g_modif : nx.Graph, nombre : int, index1 , index2):
    
    index_compteur = 0 
    collection_solutions = dict() 
    
    solution = dijkstra(g_modif, index1, index2)
    if(solution == -1):
        print('No path (supprimerJusquaNombre)')
        return -1 
    else : 
        collection_solutions[index_compteur] = solution; 
        
        memoire = []
        
        while(len(solution['path']) < nombre ):
            
            ###################################
            for i in range(len(solution['path'])-1) :
                w = g_modif.get_edge_data(solution['path'][i], solution['path'][i+1])['weight']
                memoire.append([solution['path'][i], solution['path'][i+1], w])
                
                # delete les paths le plus vite --> on trouve les autres paths plus vite 
                g_modif.remove_edge(solution['path'][i], solution['path'][i+1])
            
            ###################################
            # trouve la nouvelle solution 
            solution = dijkstra(g_modif, index1, index2)
            if(solution == -1):
                print('No path (supprimerJusquaNombre)')
                return -1 
            else : 
                collection_solutions[index_compteur] = solution; 
                # ajouter la solution au dict de resultat 
                collection_solutions[index_compteur] = solution
                # augementer le compteur ( qui fonctionne comme cle dans le dict)
                index_compteur = index_compteur + 1 

        # remettre le graph au graph originale 
        g_modif.add_weighted_edges_from(memoire)
    
    return collection_solutions    

In [276]:
def plus_courte(index1, index2, airport1, airport2): 
    if graph_totale.has_node(index1):
        if graph_totale.has_node(index2):
            
            ###################### plus courtes ###########################
             
            resultat = dijkstra(graph_totale, index1, index2)
            if(resultat == -1 ):
                print('no path (plus_courte)')
                return -1
            else : 
                s = list(map(lambda x : list_airports_codes[x], resultat['path']))
                solution_plus_courte = dict(); 
                solution_plus_courte[0] = {'distance' : resultat['distance'], 'path' : s }
                print(solution_plus_courte)
                map_correspondance(solution_plus_courte)
                return solution_plus_courte
        else : print(airport1, ' n\'est pas presente dans le graphe')
    else : print(airport2, ' n\'est pas presente dans le graphe')

In [277]:
def differents_longeurs(index1, index2, airport1, airport2, list_nombre):
    if graph_totale.has_node(index1):
        if graph_totale.has_node(index2):
            
            
            ###################### differentes longeurs ###################
            collection_solution = supprimerJusqauNombre(graph_totale, list_nombre, index1, index2)
            if( collection_solution == -1 ):
                print('No path (differents longeurs)')
                return -1
            else : 
                new_collection_solution = dict() 
                c = 0 
                for key, contenu in collection_solution.items():
                    
                    if len(contenu['path']) <= list_nombre: #and len(contenu['path']) >= int_s : 
                        tmp = []
                        for x in contenu['path'] :
                            tmp.append(list_airports_codes[x])
                        
                        new_collection_solution[c] = {'distance' : contenu['distance'], 'path' :tmp}  
                        c = c + 1 
            
                #print('Collection : ', new_collection_solution)
                
                map_correspondance(new_collection_solution)    
                
        else : print(airport1, ' n\'est pas presente dans le graphe')
    else : print(airport2, ' n\'est pas presente dans le graphe')

In [351]:
def response_widget(nom1, nom2, nombre):
    
    airport1 = filtreDictAeroports('Name', nom1)[0]
    airport2 = filtreDictAeroports('Name', nom2)[0] 

    if not(list_airports_codes.count(airport1) == 0) : 
        index1 = list_airports_codes.index(airport1)
    else : print(nom1, 'n\'est pas dans le graphe')
    if not(list_airports_codes.count(airport2) == 0) : index2 = list_airports_codes.index(airport2)   
    else : print(nom2, 'n\'est pas dans le graphe') 

    resultat = plus_courte(index1, index2, airport1, airport2)
    print(' ')
    print(resultat)
    
    if resultat != -1 and nombre >= len(resultat[0]['path']) : 
        differents_longeurs(index1, index2, airport1, airport2, nombre)
    else : print('Le nombre que vous avez choisi c\'est trop petit')

<title>LIFRADARAVION</title>
<link rel="stylesheet" href="./dashboardLayout.css">
<link rel="stylesheet" href="https://www.w3schools.com/w3css/4/w3.css">

<body>
<div class="main">
    <h1>LIFRADARAVION</h1>
    <p>Lorem ipsum dolor sit amet, consectetur adipiscing elit. 
        Vivamus in blandit leo. Nulla fringilla felis ut lorem malesuada, eget ornare ligula pharetra. Nulla sollicitudin viverra nibh, 
        ac porttitor ligula egestas nec. In eget sem ut sem finibus dignissim. Nulla eget dui vitae lorem mollis dapibus in quis velit. 
        Sed sit amet leo ullamcorper, vestibulum odio sed, congue magna. Etiam non urna pellentesque, fermentum libero sit amet, rutrum nulla. 
        Donec porta posuere sagittis. Integer tempor ullamcorper dui, sit amet varius urna fermentum in. Donec eget ex eget purus vehicula fringilla ac a ante. 
        Aenean ut imperdiet lacus, eu euismod velit. Phasellus tortor ligula, ultrices non blandit quis, fringilla sit amet quam. 
        Quisque eget lectus sed eros mollis tincidunt non sit amet sapien. Donec in risus ut ligula commodo placerat. 
        Fusce varius ac diam sed rhoncus. Donec fringilla malesuada sagittis.</p>
</div>


<div class="main">
    <h2>Graphes</h2>
    <p>
        explications sur le facon de travailler avec les graphes 
    </p>
    <img src="./graph_t.png" size = "50%">
    <br><br>
</div>

<div>
    <h2> Interactive map </h2>
    <p>
        explication sur le fonctionnement map explication sur le fonctionnement map explication sur le fonctionnement map
        explication sur le fonctionnement map explication sur le fonctionnement map explication sur le fonctionnement map
        explication sur le fonctionnement map explication sur le fonctionnement map explication sur le fonctionnement map
        explication sur le fonctionnement map explication sur le fonctionnement map explication sur le fonctionnement map 
    </p>

</div>

In [279]:
import ipyvuetify as v

In [369]:
airportD = v.Select(
    _metadata={'mount_id': 'airportD'}, 
    label = 'Airport de depart', 
    items = list_depart, 
    thumb_label='always', 
    class_='px-4', 
    v_model = "Amsterdam Schiphol Airport")

airportA = v.Select(
    _metadata={'mount_id': 'Arrivee'}, 
    label = 'Airport d\'arrivee', 
    items = list_arrivee, 
    thumb_label='always', 
    class_='px-4', 
    v_model= "Los Angeles International Airport")

nbAirports = v.Select(
    _metadata={'mount_id': 'NbAirports'}, 
    label = 'Combien de airports en totale?', 
    items = [1,2,3,4,5], 
    thumb_label='always', 
    class_='px-2', 
    v_model=2)

choix = v.Container(children = [airportD, airportA, nbAirports])

def on_change(widget, event, data):
    HTML('<p _metadata={\'mount_id\': \'NbAirports\'}> test test  </p>') 
choix.on_event('onclick', on_change)


In [319]:
from traitlets import link

In [367]:
response_widget(airportD.v_model, airportA.v_model, nbAirports.v_model)

{0: {'distance': 8950.27748845727, 'path': ['AMS', 'LAX']}}
 
{0: {'distance': 8950.27748845727, 'path': ['AMS', 'LAX']}}
Le nombre que vous avez choisi c'est trop petit


In [384]:
v.Tabs(_metadata={'mount_id': 'content-nav'}, children=[
    v.Html(tag='<div>', children=[
    ])
])

Tabs(children=[Html(tag='<div>')])

In [370]:
v.Tabs(_metadata={'mount_id': 'content-main'}, children=[
    v.Tab(children=['Interactive']),
    v.Tab(children=['Theorie']),
    v.TabItem(children=[
        v.Layout(row=True, wrap=True, align_center=True, children=[
            v.Flex(xs12=True, lg6=True, xl4=True, children=[
                choix.children[0], choix.children[1]
                
            ]),
            v.Flex(xs12=True, lg6=True, xl4=True, children=[
                choix.children[2]
                
            ]),
            v.Flex(xs12=True, lg6=True, xl4=True, children=[
                
            ]),
            v.Flex(xs12=True, xl4=True, children=[
                outputMAP
            ]),
            
            # https://vuetifyjs.com/en/styles/display/#display
        ])
    ]),
    v.TabItem(children=[
        v.Container(children=['Lorum ipsum'])
    ])
])

# code from https://github.com/voila-dashboards/voila-vuetify/blob/master/bqplot_vuetify_example.ipynb

Tabs(children=[Tab(children=['Interactive']), Tab(children=['Theorie']), TabItem(children=[Layout(align_center…

In [283]:
# drawing_graph = dessinerGraphe(graph_totale) # 55s 

In [284]:
g = ig.Graph.TupleList(graph_totale.edges(), directed=True)
layout = g.layout("large")

# fait une fois --> maintenant en mémoire 
#ig.plot(g, "graph_t.png", shape = 'triangle-up', autocurve = True)

In [285]:
n0 = list_airports_codes.index('LYS')
n1 = list_airports_codes.index('LBG')
n2 = list_airports_codes.index('FAB')
n3 = list_airports_codes.index('LAX')


w1 = graph_totale.get_edge_data(n3,n2)
w2 = graph_totale.get_edge_data(n1,n3)
#print(w1,w2)
print(graph_totale.get_edge_data(n0,n1))
print(graph_totale.get_edge_data(n1,n2))
print(graph_totale.get_edge_data(n2,n3))
print(graph_totale.get_edge_data(n0,n1)['weight'] + graph_totale.get_edge_data(n1,n2)['weight'] + graph_totale.get_edge_data(n2,n3)['weight'])

print("----------------------")
n4 = list_airports_codes.index('CDG')
n5 = list_airports_codes.index('LAX')
print(graph_totale.get_edge_data(n0,n4))
print(graph_totale.get_edge_data(n4,n5))
print(graph_totale.get_edge_data(n0,n4)['weight'] + graph_totale.get_edge_data(n4,n5)['weight'])


{'weight': 411.7061792235852}
{'weight': 344.33294832625165}
{'weight': 8752.142514126186}
9508.181641676023
----------------------
{'weight': 412.56144600362046}
{'weight': 9097.059799998087}
9509.621246001709
